# Classical CNN via Fashion MNIST Dataset

### Max Training Accuracy: 0.9575
### Max Validation Accuracy: 0.9234

-  Epoch 42/100 118/118 ━━━━━━━━━━━━━━━━━━━━ 4s 32ms/step - accuracy: 0.9653 - loss: 0.0969 - val_accuracy: 0.9225 - val_loss: 0.2710

In [ ]:
from tensorflow.keras import datasets, layers, models, callbacks, optimizers

In [ ]:
data = datasets.fashion_mnist
(X_train, y_train), (X_test, y_test) = data.load_data()

In [ ]:
model = models.Sequential(
    [
        layers.InputLayer(shape=(28, 28, 1)),
        layers.Conv2D(64, 7, activation="relu",padding='same'),
        layers.MaxPooling2D(2),
        layers.Conv2D(128, 3, activation="relu", padding='same'),
        layers.Conv2D(128, 3, activation="relu", padding='same'),
        layers.MaxPooling2D(2),
        layers.Conv2D(256, 3, activation="relu", padding='same'),
        layers.Conv2D(256, 3, activation="relu", padding='same'),
        layers.MaxPooling2D(2),
        layers.Flatten(),
        layers.BatchNormalization(),
        layers.Dense(256, activation="relu"),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(64, activation="relu"),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ]
)

model.compile(optimizer=optimizers.Adam(learning_rate=5e-3, epsilon=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],
              )


In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir=./logs --port=6006

In [ ]:
import os

root_logdir = os.path.join(os.curdir, "logs")

def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir() # example: "logs/run_2024_06_01-12_00_00"

tensorboard_callback = callbacks.TensorBoard(
    log_dir=run_logdir,
    histogram_freq=1,
    profile_batch=0
)

checkpoint_callback = callbacks.ModelCheckpoint(
    "models/my_keras_model.keras",
    save_best_only=True
)

early_stopping_callback = callbacks.EarlyStopping(
    patience=10,
    restore_best_weights=True
)

history = model.fit(
    X_train, y_train,
    batch_size=512,
    validation_data=(X_test, y_test),
    epochs=100,
    callbacks=[tensorboard_callback,
               early_stopping_callback,
               checkpoint_callback
               ]
)